# Setup

### Run Previous Script

Enable when debugging; elsewhere, just run it after running the master script.

In [1]:
# Enable when debugging; 
#%run ./1_Master_Script.ipynb

Master Script has been run successfully!


### Data

#### Pathfinder Wiki Map Layers

We're working with a GeoPackage (.gkpg) file, taken from https://github.com/pf-wikis/mapping. The file has the following layers:

borders

buildings

deserts

districts

forests

hills

ice

labels

land

mountains

rivers

roads

specials

swamps

waters


In [2]:
path = source_loc

# Commented out are unnecessary / unused layers
layers_list = [
    'borders',
    #'buildings',
    'deserts',
    #'districts',
    'forests',
    'hills',
    'ice',
    #'labels',
    'land',
    'mountains',
    'rivers',
    #'roads',
    'specials',
    'swamps',
    'waters'
]

layers_dict = dict()

for item in layers_list:

    layers_dict[item] = gpd.read_file(path + 'mapping-data.gpkg', layer = item)

#### Interactive Map

This is our equivalent to Google Maps, for finding point coordinates.

https://map.pathfinderwiki.com/#location=4.69/47.4/7.93

# Cleaning

### Construct Study Area

We're interested in multiple study areas, so let's clean that up in a dictionary.

In [3]:
study_dict = dict()

# Broken Lands
study_dict['broken_lands'] = layers_dict['borders'].loc[layers_dict['borders']['region'] == 'Broken Lands'].copy()

# Stolen Lands
study_dict['stolen_lands'] = layers_dict['borders'].loc[layers_dict['borders']['nation'] == 'Narland'].copy()

# Continent
land_list = [
    'Avistan', # Inner Sea, basically
    #'Casmaron' # To the east, some relevant settlements
]

continent = layers_dict['land'].loc[layers_dict['land']['Name'].isin(land_list)].copy()

continent = continent.dissolve()
continent_geo = continent.total_bounds
continent_geo = box(*continent_geo)

d = {'geometry': continent_geo}

study_dict['continent'] = gpd.GeoDataFrame(data = d, geometry = 'geometry', index = [0], crs = continent.crs)

In [4]:
# Now select, given the master parameter

if parameter_studyarea == 'Stolen Lands':

    chosen_dat = study_dict['stolen_lands'].copy()

elif parameter_studyarea == 'Broken Lands':

    chosen_dat = study_dict['broken_lands'].copy()

elif parameter_studyarea == 'Continent':

    chosen_dat = study_dict['continent']

### Validate

Some geometries may be invalid, and need to be fixed.

#### Diagnosis

In [5]:
def valid_diagnostic(dictionary):

    layer_list = list()
    invalid_list = list()
    total_list = list()
    
    for item in dictionary:

        layer_list.append(item)
        total_list.append(len(dictionary[item]))

        counts = dictionary[item].is_valid.value_counts().reset_index(name = 'count')
        counts = counts.loc[counts['index'] == False].reset_index(drop = True)

        if counts.empty == True:
            invalid_list.append(0)
        else:
            invalid_list.append(counts['count'][0])

    # Construct dataframe
    d = {
        'layer': layer_list,
        'invalid': invalid_list,
        'total': total_list
    }

    result = pd.DataFrame(data = d)

    return result

In [6]:
valid_diagnostic(layers_dict)

,layer,invalid,total
0,borders,0,179
1,deserts,2,95
2,forests,0,5277
3,hills,0,283
4,ice,0,8
5,land,1,2616
6,mountains,1,193
7,rivers,0,1667
8,specials,1,7
9,swamps,1,48


#### Validation
Yikes, that's not good! Thankfully, this can be repaired.

In [7]:
def validate(data):
    
    data.geometry = data.make_valid()
    
    return data

In [8]:
def validate_all(dictionary):

    new_dict = dict()

    for item in dictionary:
        
        validation_counts = dictionary[item].is_valid.value_counts().rename_axis('unique_values').reset_index(name = 'counts')
        validation_false = validation_counts.loc[validation_counts['unique_values'] == False]
        
        if len(validation_false) == 1: # This is true if any geometries were deemed invalid
            
            new_dict[item] = validate(dictionary[item])
            
        else:
            
            new_dict[item] = dictionary[item] # No need to validate!

    return new_dict

In [9]:
valid_dict = validate_all(layers_dict.copy())

valid_diagnostic(valid_dict)

,layer,invalid,total
0,borders,0,179
1,deserts,0,95
2,forests,0,5277
3,hills,0,283
4,ice,0,8
5,land,0,2616
6,mountains,0,193
7,rivers,0,1667
8,specials,0,7
9,swamps,0,48


### Remove Other Continents

We will pretty much *never* use stuff from other continents. In fact, because of how we use a very regionalized CRS later, they only cause problems via invalid geometries due to excessive stretching. So let's just remove them now.

In [10]:
inner_dict = valid_dict.copy()

# Filter by the continental study area, regardless of the study area parameter

for item in inner_dict:

    inner_dict[item] = gpd.clip(inner_dict[item],study_dict['continent']).reset_index(drop = True)

### Change CRS

The current maps are in classic EPSG:4326. The problem with this is, 4326 is in degrees, and therefore is not really good for geoprocessing in area-accurate and distance-accurate ways. So let's convert to a better one.

But what CRS to choose? This depends on what the study area we've chosen is.

**TODO**: change the CRS depending on the chosen study area!

Because we're interested only really in the Stolen Lands for now, let's try to find where in the real world The Stolen Lands is.

In [11]:
#layers_dict['borders'].explore(column = 'region')

Interesting! It seems to be in Central Europe--eastern France, southern Germany, northern Switzerland.

I found at least two interesting options:

**Option 1: Equidistant Conic**

ESRI:102031 is equidistant, meaning its distance measurements are accurate.

**Option 2: Equal Area Conic**

ESRI:102013 is Albers Equal Area Conic, meaning its area measurements are accurate.

While both area and distance are important for our purposes, we're really interested in distance. This map is about travel, not analyzing areas / densities.

In [12]:
chosen_crs = 'esri:102031'

In [13]:
crs_dict = inner_dict.copy()

for item in crs_dict:

    crs_dict[item] = crs_dict[item].to_crs(chosen_crs)

In [14]:
# Some more items to convert

for item in study_dict:

    study_dict[item] = study_dict[item].to_crs(chosen_crs)


chosen_dat = chosen_dat.to_crs(chosen_crs)

### Rivers

As it stand right now, the rivers are linestring geometries. But, for our purposes, we need them buffered!

How much should they be buffered by? Thankfully, each river is given a width we may use for buffering.

According to folks from the discord server, 'width' is only somewhat useful. It's in meters, but refers to size *in QGIS the map*. So they're of course going to be much larger than they actually are!

To be honest, I want them to be kinda big too. So I'm going to normalize the column, by making the smallest river width 10 meters, but keep the proportional difference across widths.

Also, after buffering, there should be no overlap between water and river.

**NOTE**: unresolved bug. For some reason, there is a tiny sliver leftover after clipping, causing some issues in the cost distance map.

In [15]:
river_dict = crs_dict.copy()

# Normalize rivers
new_smallest = 10
smallest_river_size = river_dict['rivers']['width'].min()
smallest_river_size = float(smallest_river_size)
normalizer = smallest_river_size/new_smallest

river_dict['rivers']['width'] = river_dict['rivers']['width']/normalizer

# Define function
def river_buffer(row):

    return row.geometry.buffer(row.width)

# Apply row-wise

river_dict['rivers']['geometry'] = river_dict['rivers'].apply(river_buffer, axis=1)

# Remove overlap between rivers and waters

not_water = gpd.clip(river_dict['waters'],study_dict['continent'])

river_dict['rivers'] = gpd.overlay(river_dict['rivers'],not_water, how = 'difference')

### Filter to Study Area

#### Bounding Box

To save on runtime, let's now filter every layer to the chosen study area.It should be a bounding box. If, for whatever reason, the chosen area is already a bounding box, then the fact that it was made into a bounding box in a prior CRS will mean the new box retains utility.

In [16]:
studyarea_geo = chosen_dat.total_bounds
studyarea_geo = box(*studyarea_geo)

d = {'geometry': studyarea_geo}

studyarea = gpd.GeoDataFrame(data = d, geometry = 'geometry', index = [0], crs = chosen_dat.crs)

#ax = studyarea.plot(color = 'darkgrey')

#chosen_dat.plot(ax=ax,color = 'pink', edgecolor = 'black')

#### Filter

In [17]:
filter_dict = river_dict.copy()

for item in filter_dict:

    filter_dict[item] = gpd.clip(filter_dict[item],studyarea).reset_index(drop = True)

### Dissolve-Explode (DISABLED)

Some of the geometries here are arbitrarily split up. Let's fix that by dissolving and exploding, which will first create one big geometry, and then split up all the unconnected ones.

Disabled for now, as I'm not yet sure if it's worth it to dissolve-explode yet.

### Lower Precision (DISABLED)

Precision is the number of decimal points each point goes to in each geometry. The higher the precision, the more bytes of data. Depending on how much precision one wants, it may actually help to reduce precision, to make things easier in future steps.

It seems like, by default, precision is 3--or 3 decimal points.

In [18]:
prec_dict = filter_dict.copy()

# DISABLED FOR NOW

#for item in prec_dict:
#
#    prec_dict[item]['geometry'] = prec_dict[item].set_precision(parameter_precision)

### Final Dictionary

#### Assignment

In [19]:
final_dict = prec_dict.copy()

#### Validation

In [20]:
valid_diagnostic(final_dict)

,layer,invalid,total
0,borders,0,9
1,deserts,0,0
2,forests,0,5
3,hills,0,5
4,ice,0,0
5,land,0,2
6,mountains,0,4
7,rivers,0,265
8,specials,0,0
9,swamps,0,1


#### Diagnostic

Let's make sure the dictionary lines up with the original data, in case a bug occurred.

In [23]:
explored_layer = 'borders'
examined_step = final_dict.copy()

ax = studyarea.explore(color = 'cyan')

# Important to compare to crs_dict, NOT layers_dict! THIS IS BECAUSE OF DIFFERENT CRS
ax = crs_dict[explored_layer].explore(m=ax, color = 'red')
ax = examined_step[explored_layer].explore(m=ax, color = 'yellow')

ax

# Run Message

This is to show key info when this script is run in another script.

In [22]:
print("Cleaning Layers Script has been run successfully!")
print(f"Chosen Study Area: {parameter_studyarea}")

Cleaning Layers Script has been run successfully!
Chosen Study Area: Stolen Lands
